# Enzymatic Constraints Enhancement of AGORA models

### Context



### Goals:

- 

In [1]:
from tqdm.auto import tqdm
from reframed.io.sbml import load_cbmodel,save_cbmodel
from cobra.io import read_sbml_model, write_sbml_model
from mewpy.cobra.util import add_enzyme_constraints,convert_gpr_to_dnf,split_isozymes
from mewpy.simulation import get_simulator
from mewpy.simulation.environment import Environment as Environment
from mewpy.util.request import retreive_gene,retreive_protein,get_smiles,brenda_query
import pandas as pd
import numpy as np
from urllib.request import urlopen
from functools import reduce
import json
from ast import literal_eval

In [2]:
filepath = "../models/non-ec/agora/Bacteroides_uniformis_ATCC_8492.xml"
model = read_sbml_model(filepath)
model2 = load_cbmodel(filepath)
organism = 'Bacteroides thetaiotaomicron'

Set parameter Username
Academic license - for non-commercial use only - expires 2024-11-11


In [3]:
model.id

'M_Bacteroides_uniformis_ATCC_8492'

In [4]:
model_id = str(model.id.strip('M_'))
print(model_id)

Bacteroides_uniformis_ATCC_8492


In [5]:
sim = get_simulator(model)
sim.set_objective("biomass")

sim2 = get_simulator(model2)
sim2.set_objective("R_biomass")

In [6]:
sim.simulate()

objective: 71.40464067328286
Status: OPTIMAL
Method:SimulationMethod.FBA

In [7]:
sim2.simulate()

objective: None
Status: INF_OR_UNB
Method:SimulationMethod.FBA

## Annotation scraping

In [8]:
ls_ge = []

for ge in sim.genes:
    i = sim.genes.index(ge)
    gene = sim.genes[i]
    rxns = sim.get_gene(ge).reactions
    for rx in rxns:
        anno = sim.get_reaction(rx)['annotations']
        seed_id = anno.get('seed.reactions')
        rxn_name = sim.get_reaction(rx).name
        ecnumber = anno.get('ec-code')
        metanetx = anno.get('metanetx.reaction')
        kegg = anno.get('kegg.reaction')
        res = [gene,rx,rxn_name,seed_id,metanetx,kegg,ecnumber]
        ls_ge.append(res)

df_ge = pd.DataFrame(ls_ge,columns=[['Gene','Reaction','Name','ModelSEED_id','MetaNetX','KEGG_id','ecNumber']])

df_ge

,Gene,Reaction,Name,ModelSEED_id,MetaNetX,KEGG_id,ecNumber
0,411479.1.3062_56.peg,1HIBUP_S_GLCAASE,1-hydroxy S-ibuprofen beta-glucuronidase,None,None,None,None
1,411479.1.3062_56.peg,1HMDGLUC_GLCAASE,1-OH-midazolam-glucuronide beta-glucuronidase,None,None,None,None
2,411479.1.3062_56.peg,2HATVACIDGLUC_GLCAASE,2-hydroxy-atorvastatin-acyl-glucuronide beta-g...,None,None,None,None
3,411479.1.3062_56.peg,2HATVLACGLUC_GLCAASE,2-hydroxy-atorvastatin-lactone-glucuronide bet...,None,None,None,None
4,411479.1.3062_56.peg,2HIBUP_S_GLCAASE,2-hydroxy-S-ibuprofen beta-glucuronidase,None,None,None,None
...,...,...,...,...,...,...,...
1717,411479.1.peg.2102,rtranscription,RNA transcription c0,rxn13784,None,None,None
1718,411479.1.peg.3030,rtranscription,RNA transcription c0,rxn13784,None,None,None
1719,411479.1.peg.3645,rtranscription,RNA transcription c0,rxn13784,None,None,None
1720,411479.1.peg.3749,rtranscription,RNA transcription c0,rxn13784,None,None,None


In [9]:
mask = df_ge['ecNumber'].values!=None
ec_nona = df_ge[mask]
ec_nona

,Gene,Reaction,Name,ModelSEED_id,MetaNetX,KEGG_id,ecNumber
170,411479.1.peg.1814,23DHMPO,"(R)-2,3-Dihydroxy-3-methylpentanoate:NADP+ oxi...",rxn03435,MNXR83171,None,1.1.1.86
171,411479.1.peg.1814,2AHBUTI,(S)-2-Aceto-2-hydroxybutanoate isomerase,rxn03436,MNXR76597,None,"1.1.1.86, 5.4.99.3"
172,411479.1.peg.1439,23PDE2,"2,3-Cyclic UMP 3-nucleotidohydrolase",rxn02522,MNXR34,None,3.1.4.16
173,411479.1.peg.1439,23PDE4,"2,3-Cyclic CMP 3-nucleotidohydrolase",rxn02762,MNXR35,None,3.1.4.16
174,411479.1.peg.1439,23PDE7,"2,3-Cyclic AMP 3-nucleotidohydrolase",rxn02521,MNXR36,None,3.1.4.16
...,...,...,...,...,...,...,...
1707,411479.1.peg.3150,r0775,"Formamidopyrimidine Nucleoside Triphosphate 7,...",rxn03419,MNXR85297,None,3.5.4.16
1708,411479.1.peg.3150,r0777,"GTP 7, 8-8, 9-Dihydrolase",rxn03421,MNXR74388,None,3.5.4.16
1709,411479.1.1966_16.peg,r0127,L-Asparagine Amidohydrolase / Cyanoamino Acid ...,rxn00342,MNXR81754,None,3.5.1.38
1710,411479.1.2441_16.peg,r0127,L-Asparagine Amidohydrolase / Cyanoamino Acid ...,rxn00342,MNXR81754,None,3.5.1.38


In [10]:
seed_id = df_ge['ModelSEED_id'].values.tolist()

seed_id = [reduce(lambda x: x, inner_list) for inner_list in seed_id]

metanetx_id = df_ge['MetaNetX'].values.tolist()

metanetx_id = [reduce(lambda x: x, inner_list) for inner_list in metanetx_id]

kegg_id = df_ge['KEGG_id'].values.tolist()

kegg_id = [reduce(lambda x: x, inner_list) for inner_list in kegg_id]

 ## ModelSEED query

In [11]:
SOLR_URL='https://modelseed.org'
ls_name = []
ls_kegg = []
ls_bigg = []

for mseed_id in tqdm(seed_id):
    i = seed_id.index(mseed_id)
    if mseed_id == None:
        ls_name.append('None')
        ls_kegg.append('None')
        ls_bigg.append('None')
    else:
        try:
            connection = urlopen(SOLR_URL+f'/solr/reactions/select?wt=json&q=id:{mseed_id}&fl=name,id,formula,charge,aliases')
            response = json.load(connection)
            for document in response['response']['docs']:  
                ms_name = document.get('name')
                ls_alias = document.get('aliases')
                ms_bigg = list(filter(lambda a: 'BiGG:' in a, document.get('aliases')))
                ms_kegg = list(filter(lambda a: 'KEGG:' in a, document.get('aliases')))
                if len(ms_bigg)== 0 and len(ms_kegg)== 0:
                    ms_bigg = 'None'
                    ms_kegg = 'None'
                elif len(ms_bigg)== 0 and len(ms_kegg)!= 0:
                    ms_bigg = 'None'
                    ms_kegg = list(ms_kegg)[0]
                    ms_kegg = ms_kegg.replace('KEGG: ','')
                elif len(ms_bigg)!= 0 and len(ms_kegg)== 0:
                    ms_kegg = 'None'
                    ms_bigg = list(ms_bigg)[0]
                    ms_bigg = ms_bigg.replace('BiGG: ','')
                else:
                    ms_kegg = list(ms_kegg)[0]
                    ms_kegg = ms_kegg.replace('KEGG: ','')
                    ms_bigg = list(ms_bigg)[0]
                    ms_bigg = ms_bigg.replace('BiGG: ','')    
                ls_name.append(ms_name)
                ls_bigg.append(ms_bigg)
                ls_kegg.append(ms_kegg)           
        except:
            ls_name.append('None')
            ls_kegg.append('None')
            ls_bigg.append('None')

  0%|          | 0/1722 [00:00<?, ?it/s]

In [12]:
new_kegg = [next(filter(None, i)) for i in zip(ls_kegg, kegg_id)]

In [13]:
df_ge['BIGG_id'] = ls_bigg
df_ge['KEGG_id'] = new_kegg

## BiGG query

In [14]:
import requests

ls_bigg = df_ge['BIGG_id'].values.tolist()
bigg_ls = []

for bigg in tqdm(ls_bigg):
    for bigg_n in bigg:
        bigg_n = str(bigg_n)
        bigg_n = bigg_n.split(';')
        bigg_n = [x.strip(' ') for x in bigg_n]
        sub_bigg_ls = []
        for bi in bigg_n:
            if bi == 'None':
                pass
            else:
                url =f'http://bigg.ucsd.edu/api/v2/universal/reactions/{bi}'
                with requests.request("GET", url) as resp:
                    try:
                        resp.raise_for_status()  # raises exception when not a 2xx response
                        if resp.status_code != 204:
                            data = dict(resp.json())
                            ec_l = data['database_links']
                            if ec_l == None:
                                sub_bigg_ls.append(None)
                            else:
                                ec = [i['id'] for i in ec_l['EC Number']]
                                if ec == None:
                                    sub_bigg_ls.append(None)
                                sub_bigg_ls.append(ec)
                        else: 
                            sub_bigg_ls.append(None)
                    except:
                        sub_bigg_ls.append(None)
    bigg_ls.append(sub_bigg_ls)

  0%|          | 0/1722 [00:00<?, ?it/s]

In [15]:
ec_l = df_ge['ecNumber'].values.tolist()
new_l = [next(filter(None, i)) for i in zip(bigg_ls, ec_l)]
df_ge['ecNumber'] = new_l

In [16]:
df_ge

,Gene,Reaction,Name,ModelSEED_id,MetaNetX,KEGG_id,ecNumber,BIGG_id
0,411479.1.3062_56.peg,1HIBUP_S_GLCAASE,1-hydroxy S-ibuprofen beta-glucuronidase,None,None,None,[None],None
1,411479.1.3062_56.peg,1HMDGLUC_GLCAASE,1-OH-midazolam-glucuronide beta-glucuronidase,None,None,None,[None],None
2,411479.1.3062_56.peg,2HATVACIDGLUC_GLCAASE,2-hydroxy-atorvastatin-acyl-glucuronide beta-g...,None,None,None,[None],None
3,411479.1.3062_56.peg,2HATVLACGLUC_GLCAASE,2-hydroxy-atorvastatin-lactone-glucuronide bet...,None,None,None,[None],None
4,411479.1.3062_56.peg,2HIBUP_S_GLCAASE,2-hydroxy-S-ibuprofen beta-glucuronidase,None,None,None,[None],None
...,...,...,...,...,...,...,...,...
1717,411479.1.peg.2102,rtranscription,RNA transcription c0,rxn13784,None,None,[None],None
1718,411479.1.peg.3030,rtranscription,RNA transcription c0,rxn13784,None,None,[None],None
1719,411479.1.peg.3645,rtranscription,RNA transcription c0,rxn13784,None,None,[None],None
1720,411479.1.peg.3749,rtranscription,RNA transcription c0,rxn13784,None,None,[None],None


## Substrates

In [17]:
rx_l = df_ge['Reaction'].values.tolist()
ls_sub = []

for rxn in rx_l:
    for rx in rxn:
        sub = list(sim.get_substrates(rx).keys())
        ls_sub.append(sub)
    
df_ge["Substrates"] = ls_sub   

df_ge

,Gene,Reaction,Name,ModelSEED_id,MetaNetX,KEGG_id,ecNumber,BIGG_id,Substrates
0,411479.1.3062_56.peg,1HIBUP_S_GLCAASE,1-hydroxy S-ibuprofen beta-glucuronidase,None,None,None,[None],None,"[1hibupglu_S[c], h2o[c]]"
1,411479.1.3062_56.peg,1HMDGLUC_GLCAASE,1-OH-midazolam-glucuronide beta-glucuronidase,None,None,None,[None],None,"[1hmdgluc[c], h2o[c]]"
2,411479.1.3062_56.peg,2HATVACIDGLUC_GLCAASE,2-hydroxy-atorvastatin-acyl-glucuronide beta-g...,None,None,None,[None],None,"[2hatvacidgluc[c], h2o[c]]"
3,411479.1.3062_56.peg,2HATVLACGLUC_GLCAASE,2-hydroxy-atorvastatin-lactone-glucuronide bet...,None,None,None,[None],None,"[2hatvlacgluc[c], h2o[c]]"
4,411479.1.3062_56.peg,2HIBUP_S_GLCAASE,2-hydroxy-S-ibuprofen beta-glucuronidase,None,None,None,[None],None,"[2hibupglu_S[c], h2o[c]]"
...,...,...,...,...,...,...,...,...,...
1717,411479.1.peg.2102,rtranscription,RNA transcription c0,rxn13784,None,None,[None],None,[]
1718,411479.1.peg.3030,rtranscription,RNA transcription c0,rxn13784,None,None,[None],None,[]
1719,411479.1.peg.3645,rtranscription,RNA transcription c0,rxn13784,None,None,[None],None,[]
1720,411479.1.peg.3749,rtranscription,RNA transcription c0,rxn13784,None,None,[None],None,[]


In [18]:
sub_na = df_ge['Substrates'].values.tolist()

ls_sub = []
ls_smile =[]



for sub_l in tqdm(sub_na):
    sub_ls_sub = []
    sub_ls_smile = []
    for sub_s in sub_l:
        for sub in sub_s:
            sub_name = sim.get_metabolite(sub).get('name')
            if "2,3-C" in sub_name:
                sub_name = sub_name.replace("2,3","2',3'")
            elif "2,3-c" in sub_name:
                sub_name = sub_name.replace("2,3","2',3'")
            elif "3-triphosphate" in sub_name:
                sub_name = sub_name.replace("3","3'")
            else:
                pass
            smile = get_smiles(sub_name)
            sub_ls_smile.append(smile)
            sub_ls_sub.append(sub_name)
        ls_sub.append(sub_ls_sub)
        ls_smile.append(sub_ls_smile)

df_ge['Substrate Name'] = ls_sub
df_ge['Substrate SMILES'] = ls_smile
df_ge.to_csv(f'../data/ec_data/ge_data_{model_id}.csv',na_rep='None')

  0%|          | 0/1722 [00:00<?, ?it/s]

In [19]:
df_ge = pd.read_csv(f'../data/ec_data/ge_data_{model_id}.csv')
df_ge = df_ge.loc[:,['Gene','Reaction','Name','ModelSEED_id','MetaNetX','KEGG_id','BIGG_id','Substrates','Substrate Name','Substrate SMILES','ecNumber']]
df_ge

,Gene,Reaction,Name,ModelSEED_id,MetaNetX,KEGG_id,BIGG_id,Substrates,Substrate Name,Substrate SMILES,ecNumber
0,411479.1.3062_56.peg,1HIBUP_S_GLCAASE,1-hydroxy S-ibuprofen beta-glucuronidase,None,None,None,None,"['1hibupglu_S[c]', 'h2o[c]']","['1-hydroxy S-ibuprofen-glucuronide', 'Water']","[None, 'O']",[None]
1,411479.1.3062_56.peg,1HMDGLUC_GLCAASE,1-OH-midazolam-glucuronide beta-glucuronidase,None,None,None,None,"['1hmdgluc[c]', 'h2o[c]']","['1-OH-midazolam-glucuronide', 'Water']","[None, 'O']",[None]
2,411479.1.3062_56.peg,2HATVACIDGLUC_GLCAASE,2-hydroxy-atorvastatin-acyl-glucuronide beta-g...,None,None,None,None,"['2hatvacidgluc[c]', 'h2o[c]']","['2-hydroxy-atorvastatin-acyl-glucuronide', 'W...","[None, 'O']",[None]
3,411479.1.3062_56.peg,2HATVLACGLUC_GLCAASE,2-hydroxy-atorvastatin-lactone-glucuronide bet...,None,None,None,None,"['2hatvlacgluc[c]', 'h2o[c]']","['2-hydroxy-atorvastatin-lactone-glucuronide',...","[None, 'O']",[None]
4,411479.1.3062_56.peg,2HIBUP_S_GLCAASE,2-hydroxy-S-ibuprofen beta-glucuronidase,None,None,None,None,"['2hibupglu_S[c]', 'h2o[c]']","['2-hydroxy S-ibuprofen-glucuronide', 'Water']","[None, 'O']",[None]
...,...,...,...,...,...,...,...,...,...,...,...
1717,411479.1.peg.2102,rtranscription,RNA transcription c0,rxn13784,None,None,None,[],[],[],[None]
1718,411479.1.peg.3030,rtranscription,RNA transcription c0,rxn13784,None,None,None,[],[],[],[None]
1719,411479.1.peg.3645,rtranscription,RNA transcription c0,rxn13784,None,None,None,[],[],[],[None]
1720,411479.1.peg.3749,rtranscription,RNA transcription c0,rxn13784,None,None,None,[],[],[],[None]


In [20]:
ec_l = df_ge['ecNumber'].values.tolist()
ec_nl = []
ei = []

for es in ec_l:
    es = str(es)
    es = es.split(',')
    ei = []
    for sublist in es:
        sublist = str(sublist)
        sublist = sublist.strip("[[")
        sublist = sublist.strip(" ")
        sublist = sublist.strip("'")
        sublist = sublist.strip("[")
        sublist = sublist.strip("[")
        sublist = sublist.strip("'")
        sublist = sublist.strip("'")
        sublist = sublist.strip("]")
        sublist = sublist.strip("]]")  
        sublist = sublist.strip("'")
        if sublist not in ei:
            if sublist == 'None' and len(ei) > 0:
                pass
            elif '-' in sublist:
                pass
            else:
                ei.append(sublist)
    ec_nl.append(ei)

df_ge['ecNumber'] = ec_nl

## BRENDA query

### Kcat extraction

In [21]:
from brendapyrser import BRENDA
from brendapyrser import EnzymePropertyDict


dataFile = '../../brenda_2023_1.txt'

In [22]:
brenda = BRENDA(dataFile)
brenda

Number of Enzymes,7832
BRENDA copyright,"Copyrighted by Dietmar Schomburg, Techn. University Braunschweig, GERMANY. Distributed under the License as stated at http:/www.brenda-enzymes.org"
Parser version,0.0.1
Author,"Semidán Robaina Estévez, 2020"


In [23]:
kcat_ls = []
ec_ls = df_ge['ecNumber'].values.tolist()

for ec in tqdm(ec_ls):
    ec=str(ec)
    ec = ec.split(',')
    sub_kcat_ls = []
    #print(ec)
    for ec_n in ec:
        ec_n = ec_n.strip(',')
        ec_n = ec_n.strip('[')
        ec_n = ec_n.strip('[[')
        ec_n = ec_n.strip(']')
        ec_n = ec_n.strip(']]')
        ec_n = ec_n.strip(' ')
        ec_n = ec_n.strip('[')
        ec_n = ec_n.strip("'")
        ec_n = ec_n.strip(',')
        ec_n = ec_n.strip(' ')
        if "-" in ec_n:
            pass
        elif ec_n == None:
            sub_kcat_ls.append(None)
            continue
        else:
            try:
                r = brenda.reactions.get_by_id(ec_n)
                kcat_va = r.Kcatvalues.get_values()
                avg_kcat = sum(kcat_va)/len(kcat_va)
                sub_kcat_ls.append(avg_kcat)
            except:
                sub_kcat_ls.append(None)
    kcat_ls.append(sub_kcat_ls)

df_ge['Avg Kcat (by ec)'] = kcat_ls

  0%|          | 0/1722 [00:00<?, ?it/s]

In [24]:
kcat_ls = []
ec_ls = df_ge['ecNumber'].values.tolist()


for ec in tqdm(ec_ls):
    ec=str(ec)
    ec = ec.split(',')
    sub_kcat_ls = []
    for ec_n in ec:
        ec_n = ec_n.strip(',')
        ec_n = ec_n.strip('[')
        ec_n = ec_n.strip('[[')
        ec_n = ec_n.strip(']')
        ec_n = ec_n.strip(']]')
        ec_n = ec_n.strip(' ')
        ec_n = ec_n.strip('[')
        ec_n = ec_n.strip("'")
        if "-" in ec_n:
            pass
        else:
            try:
                r = brenda.reactions.get_by_id(ec_n)
                kcat_va = r.Kcatvalues.filter_by_organism(organism).get_values()
                avg_kcat = sum(kcat_va)/len(kcat_va)
                sub_kcat_ls.append(avg_kcat)
            except:
                sub_kcat_ls.append(None)
    kcat_ls.append(sub_kcat_ls)

df_ge['Avg Kcat (by ec and species)'] = kcat_ls

  0%|          | 0/1722 [00:00<?, ?it/s]

### Sequence extraction

In [25]:
seq_ls = []
ec_ls = df_ge['ecNumber'].values.tolist()


for ec in tqdm(ec_ls):
    ec=str(ec)
    ec = ec.split(',')
    sub_seq_ls = []
    for ec_n in ec:
        ec_n = ec_n.strip(',')
        ec_n = ec_n.strip('[')
        ec_n = ec_n.strip('[[')
        ec_n = ec_n.strip(']')
        ec_n = ec_n.strip(']]')
        ec_n = ec_n.strip(' ')
        ec_n = ec_n.strip('[')
        ec_n = ec_n.strip("'")
        ec_n = ec_n.strip(' ')
        ec_n = ec_n.strip('[')
        if "-" in ec_n:
            pass
        elif ec_n == None:
            sub_mw_ls.append(None)
        else:
            try:
                from zeep import Client
                import hashlib

                wsdl = "https://www.brenda-enzymes.org/soap/brenda_zeep.wsdl"
                password = hashlib.sha256("07042000Alex!".encode("utf-8")).hexdigest()
                client = Client(wsdl)
                parameters = ( "alexandreareias1718@gmail.com",password,f"ecNumber*{ec_n}","sequence*", "noOfAminoAcids*", "firstAccessionCode*", "source*", "id*", "organism*Bacteroides sp")
                resultString = client.service.getSequence(*parameters) 
                sub_seq_ls.append(resultString[0]['sequence'])
            except:
                sub_seq_ls.append(None)                  
    seq_ls.append(sub_seq_ls)
    
df_ge['Protein Sequence'] = seq_ls

  0%|          | 0/1722 [00:00<?, ?it/s]

### Molecular Weight extraction

In [26]:
mw_ls = []
ec_ls = df_ge['ecNumber'].values.tolist()


for ec in tqdm(ec_ls):
    ec=str(ec)
    ec = ec.split(',')
    sub_mw_ls = []
    for ec_n in ec:
        ec_n = ec_n.strip(',')
        ec_n = ec_n.strip('[')
        ec_n = ec_n.strip('[[')
        ec_n = ec_n.strip(']')
        ec_n = ec_n.strip(']]')
        ec_n = ec_n.strip(' ')
        ec_n = ec_n.strip('[')
        ec_n = ec_n.strip("'")
        ec_n = ec_n.strip(' ')
        ec_n = ec_n.strip('[')
        if "-" in ec_n:
            pass
        elif ec_n == None:
            sub_mw_ls.append(None)
        else:
            try:
                from zeep import Client
                import hashlib
                
                res_mw = 0
                wsdl = "https://www.brenda-enzymes.org/soap/brenda_zeep.wsdl"
                password = hashlib.sha256("07042000Alex!".encode("utf-8")).hexdigest()
                client = Client(wsdl)
                parameters = ("alexandreareias1718@gmail.com",password,f"ecNumber*{ec_n}","molecularWeight*","molecularWeightMaximum*","commentary*","organism*","literature*" )
                resultString = client.service.getMolecularWeight(*parameters)
                for i in range(len(resultString)):
                    res_mw = res_mw + int(resultString[i]['molecularWeight'])
                res_mw = res_mw/len(resultString)
                sub_mw_ls.append(res_mw)
            except:
                sub_mw_ls.append(None)                  
    mw_ls.append(sub_mw_ls)

df_ge['Molecular Weight'] = mw_ls

  0%|          | 0/1722 [00:00<?, ?it/s]

In [28]:
from zeep import Client
import hashlib

res_mw = 0

wsdl = "https://www.brenda-enzymes.org/soap/brenda_zeep.wsdl"
password = hashlib.sha256("07042000Alex!".encode("utf-8")).hexdigest()
client = Client(wsdl)
parameters = ("alexandreareias1718@gmail.com",password,"ecNumber*1.1.1.86", "molecularWeight*", "molecularWeightMaximum*", "commentary*", "organism*", "literature*")
resultString = client.service.getMolecularWeight(*parameters) 
for i in range(len(resultString)):
        res_mw = res_mw + int(resultString[i]['molecularWeight'])
res_mw = res_mw/len(resultString)
print(res_mw)

138656.35714285713


In [29]:
df_ge

,Gene,Reaction,Name,ModelSEED_id,MetaNetX,KEGG_id,BIGG_id,Substrates,Substrate Name,Substrate SMILES,ecNumber,Avg Kcat (by ec),Avg Kcat (by ec and species),Protein Sequence,Molecular Weight
0,411479.1.3062_56.peg,1HIBUP_S_GLCAASE,1-hydroxy S-ibuprofen beta-glucuronidase,None,None,None,None,"['1hibupglu_S[c]', 'h2o[c]']","['1-hydroxy S-ibuprofen-glucuronide', 'Water']","[None, 'O']",[None],[None],[None],[None],[None]
1,411479.1.3062_56.peg,1HMDGLUC_GLCAASE,1-OH-midazolam-glucuronide beta-glucuronidase,None,None,None,None,"['1hmdgluc[c]', 'h2o[c]']","['1-OH-midazolam-glucuronide', 'Water']","[None, 'O']",[None],[None],[None],[None],[None]
2,411479.1.3062_56.peg,2HATVACIDGLUC_GLCAASE,2-hydroxy-atorvastatin-acyl-glucuronide beta-g...,None,None,None,None,"['2hatvacidgluc[c]', 'h2o[c]']","['2-hydroxy-atorvastatin-acyl-glucuronide', 'W...","[None, 'O']",[None],[None],[None],[None],[None]
3,411479.1.3062_56.peg,2HATVLACGLUC_GLCAASE,2-hydroxy-atorvastatin-lactone-glucuronide bet...,None,None,None,None,"['2hatvlacgluc[c]', 'h2o[c]']","['2-hydroxy-atorvastatin-lactone-glucuronide',...","[None, 'O']",[None],[None],[None],[None],[None]
4,411479.1.3062_56.peg,2HIBUP_S_GLCAASE,2-hydroxy-S-ibuprofen beta-glucuronidase,None,None,None,None,"['2hibupglu_S[c]', 'h2o[c]']","['2-hydroxy S-ibuprofen-glucuronide', 'Water']","[None, 'O']",[None],[None],[None],[None],[None]
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1717,411479.1.peg.2102,rtranscription,RNA transcription c0,rxn13784,None,None,None,[],[],[],[None],[None],[None],[None],[None]
1718,411479.1.peg.3030,rtranscription,RNA transcription c0,rxn13784,None,None,None,[],[],[],[None],[None],[None],[None],[None]
1719,411479.1.peg.3645,rtranscription,RNA transcription c0,rxn13784,None,None,None,[],[],[],[None],[None],[None],[None],[None]
1720,411479.1.peg.3749,rtranscription,RNA transcription c0,rxn13784,None,None,None,[],[],[],[None],[None],[None],[None],[None]


## DLKcat - Kcat prediction

In [30]:
dk_prep = df_ge.drop(columns=['Substrates','ModelSEED_id','MetaNetX','KEGG_id','BIGG_id','ecNumber'])
dk_prep

,Gene,Reaction,Name,Substrate Name,Substrate SMILES,Avg Kcat (by ec),Avg Kcat (by ec and species),Protein Sequence,Molecular Weight
0,411479.1.3062_56.peg,1HIBUP_S_GLCAASE,1-hydroxy S-ibuprofen beta-glucuronidase,"['1-hydroxy S-ibuprofen-glucuronide', 'Water']","[None, 'O']",[None],[None],[None],[None]
1,411479.1.3062_56.peg,1HMDGLUC_GLCAASE,1-OH-midazolam-glucuronide beta-glucuronidase,"['1-OH-midazolam-glucuronide', 'Water']","[None, 'O']",[None],[None],[None],[None]
2,411479.1.3062_56.peg,2HATVACIDGLUC_GLCAASE,2-hydroxy-atorvastatin-acyl-glucuronide beta-g...,"['2-hydroxy-atorvastatin-acyl-glucuronide', 'W...","[None, 'O']",[None],[None],[None],[None]
3,411479.1.3062_56.peg,2HATVLACGLUC_GLCAASE,2-hydroxy-atorvastatin-lactone-glucuronide bet...,"['2-hydroxy-atorvastatin-lactone-glucuronide',...","[None, 'O']",[None],[None],[None],[None]
4,411479.1.3062_56.peg,2HIBUP_S_GLCAASE,2-hydroxy-S-ibuprofen beta-glucuronidase,"['2-hydroxy S-ibuprofen-glucuronide', 'Water']","[None, 'O']",[None],[None],[None],[None]
...,...,...,...,...,...,...,...,...,...
1717,411479.1.peg.2102,rtranscription,RNA transcription c0,[],[],[None],[None],[None],[None]
1718,411479.1.peg.3030,rtranscription,RNA transcription c0,[],[],[None],[None],[None],[None]
1719,411479.1.peg.3645,rtranscription,RNA transcription c0,[],[],[None],[None],[None],[None]
1720,411479.1.peg.3749,rtranscription,RNA transcription c0,[],[],[None],[None],[None],[None]


In [31]:
dk_prep['Substrate Name'] = dk_prep['Substrate Name'].apply(literal_eval) #convert to list type
dk_prep['Substrate SMILES'] = dk_prep['Substrate SMILES'].apply(literal_eval) #convert to list type
dk_prep = dk_prep.explode(['Substrate Name','Substrate SMILES']).reset_index(drop=True)

In [32]:
#dk_prep['Protein Sequence'] = dk_prep['Protein Sequence'].apply(literal_eval) #convert to list type
#dk_prep['Molecular Weight'] = dk_prep['Molecular Weight'].apply(literal_eval) #convert to list type
dk_prep = dk_prep.explode(['Protein Sequence','Molecular Weight','Avg Kcat (by ec)','Avg Kcat (by ec and species)']).reset_index(drop=True)

In [33]:
dk_inp = dk_prep.drop(columns=['Gene','Reaction', 'Name', 'Avg Kcat (by ec)', 'Avg Kcat (by ec and species)','Molecular Weight'])
dk_inp.to_csv(f'../../DLKcat/DeeplearningApproach/Code/example/dk_input_{model_id}.tsv',sep="\t",na_rep='None',index= False)

In [34]:
dk_inp

,Substrate Name,Substrate SMILES,Protein Sequence
0,1-hydroxy S-ibuprofen-glucuronide,None,None
1,Water,O,None
2,1-OH-midazolam-glucuronide,None,None
3,Water,O,None
4,2-hydroxy-atorvastatin-acyl-glucuronide,None,None
...,...,...,...
4727,NaN,NaN,NaN
4728,NaN,NaN,NaN
4729,NaN,NaN,NaN
4730,NaN,NaN,NaN


Run DLKcat

In [35]:
dk_out = pd.read_csv(f'../../DLKcat/DeeplearningApproach/Code/example/output.tsv', sep="\t")
dk_out['Gene'] = dk_prep['Gene']
dk_out['Reaction'] = dk_prep['Reaction']
dk_out['Molecular Weight'] = dk_prep['Molecular Weight']
dk_out['Avg Kcat (by ec)'] = dk_prep['Avg Kcat (by ec)']
dk_out = dk_out.loc[:,['Gene','Reaction','Substrate Name','Substrate SMILES','Protein Sequence','Molecular Weight','Kcat value (1/s)','Avg Kcat (by ec)']]  
dk_out

,Gene,Reaction,Substrate Name,Substrate SMILES,Protein Sequence,Molecular Weight,Kcat value (1/s),Avg Kcat (by ec)
0,411479.1.3062_56.peg,1HIBUP_S_GLCAASE,1-hydroxy S-ibuprofen-glucuronide,None,None,None,None,None
1,411479.1.3062_56.peg,1HIBUP_S_GLCAASE,Water,O,None,None,0.8537,None
2,411479.1.3062_56.peg,1HMDGLUC_GLCAASE,1-OH-midazolam-glucuronide,None,None,None,None,None
3,411479.1.3062_56.peg,1HMDGLUC_GLCAASE,Water,O,None,None,0.8537,None
4,411479.1.3062_56.peg,2HATVACIDGLUC_GLCAASE,2-hydroxy-atorvastatin-acyl-glucuronide,None,None,None,None,None
...,...,...,...,...,...,...,...,...
4727,411479.1.peg.2102,rtranscription,None,None,None,None,None,None
4728,411479.1.peg.3030,rtranscription,None,None,None,None,None,None
4729,411479.1.peg.3645,rtranscription,None,None,None,None,None,None
4730,411479.1.peg.3749,rtranscription,None,None,None,None,None,None


### Joining Kcat values

In [36]:
kcat_brenda = dk_out['Avg Kcat (by ec)'].values.tolist()
kcat_dl = dk_out['Kcat value (1/s)'].values.tolist()
new_kcat = []

for i in range(len(kcat_brenda)):
    if kcat_brenda[i] != None and kcat_dl[i] != 'None':
        kcat = (float(kcat_brenda[i]) + float(kcat_dl[i]))/2
    elif kcat_brenda[i] == None and kcat_dl[i] != 'None':
        kcat = kcat_dl[i]
    elif kcat_brenda[i] != None and kcat_dl[i] == 'None':
        kcat = kcat_brenda[i]
    elif kcat_brenda[i] == None and kcat_dl[i] == 'None':
        kcat = 0
    new_kcat.append(kcat)
    
dk_out['New Kcat'] = new_kcat
dk_out

,Gene,Reaction,Substrate Name,Substrate SMILES,Protein Sequence,Molecular Weight,Kcat value (1/s),Avg Kcat (by ec),New Kcat
0,411479.1.3062_56.peg,1HIBUP_S_GLCAASE,1-hydroxy S-ibuprofen-glucuronide,None,None,None,None,None,0
1,411479.1.3062_56.peg,1HIBUP_S_GLCAASE,Water,O,None,None,0.8537,None,0.8537
2,411479.1.3062_56.peg,1HMDGLUC_GLCAASE,1-OH-midazolam-glucuronide,None,None,None,None,None,0
3,411479.1.3062_56.peg,1HMDGLUC_GLCAASE,Water,O,None,None,0.8537,None,0.8537
4,411479.1.3062_56.peg,2HATVACIDGLUC_GLCAASE,2-hydroxy-atorvastatin-acyl-glucuronide,None,None,None,None,None,0
...,...,...,...,...,...,...,...,...,...
4727,411479.1.peg.2102,rtranscription,None,None,None,None,None,None,0
4728,411479.1.peg.3030,rtranscription,None,None,None,None,None,None,0
4729,411479.1.peg.3645,rtranscription,None,None,None,None,None,None,0
4730,411479.1.peg.3749,rtranscription,None,None,None,None,None,None,0


## Adding ezymatic constraints

In [37]:
genes = dk_out['Gene'].values.tolist()
mweights = dk_out['Molecular Weight'].values.tolist()
kcats = dk_out['New Kcat'].values.tolist()

ec_data = dict()
for gene in genes:
    ge = sim.get_gene(gene).id
    i = genes.index(gene)
    mw = mweights[i]
    if mw == None:
        mw = 0
    kcat = kcats[i]
    if kcat == None:
        kcat = 1
    ec_data[ge]={'protein':ge[len(sim._g_prefix):],'mw':mw,'kcat':kcat}
    
print(ec_data)

{'411479.1.3062_56.peg': {'protein': '411479.1.3062_56.peg', 'mw': 0, 'kcat': 0}, '411479.1.peg.1814': {'protein': '411479.1.peg.1814', 'mw': 138656.35714285713, 'kcat': 2.0857631200680267}, '411479.1.peg.1439': {'protein': '411479.1.peg.1439', 'mw': 77195.8, 'kcat': 5.361464705882353}, '411479.1.peg.2180': {'protein': '411479.1.peg.2180', 'mw': 77195.8, 'kcat': 5.361464705882353}, '411479.1.3740.peg': {'protein': '411479.1.3740.peg', 'mw': 62275.0, 'kcat': 9.904871428571429}, '411479.1.821.peg': {'protein': '411479.1.821.peg', 'mw': 0, 'kcat': '1.1192'}, '411479.1.958.peg': {'protein': '411479.1.958.peg', 'mw': 0, 'kcat': '1.1192'}, '411479.1.peg.951': {'protein': '411479.1.peg.951', 'mw': 44816.291666666664, 'kcat': 6.808541666666667}, '411479.1.peg.86': {'protein': '411479.1.peg.86', 'mw': 229477.23076923078, 'kcat': 6.6908521875}, '411479.1.1307.peg': {'protein': '411479.1.1307.peg', 'mw': 104144.3870967742, 'kcat': 20.44836515151515}, '411479.1.peg.2497': {'protein': '411479.1.peg

In [38]:
import json

with open("../data/ec_data/{model_id}.json", "w") as f:
        json.dump(ec_data,f)

In [39]:
print("Genes with missing data:")
for k,v in ec_data.items():
    if v['mw']==0:
        print(k)

Genes with missing data:
411479.1.3062_56.peg
411479.1.821.peg
411479.1.958.peg
411479.1.1231.peg
411479.1.2100.peg
411479.1.peg.2438
411479.1.peg.3415
411479.1.peg.1956
411479.1.peg.2370
411479.1.peg.2854
411479.1.peg.2347
411479.1.peg.2583
411479.1.peg.2744
411479.1.peg.2891
411479.10.peg.739
411479.1.2801_33.peg
411479.1.307_22.peg
411479.1.peg.2795
411479.1.2797_37.peg
411479.1.peg.1827
411479.1.peg.1294
411479.10.peg.1516
411479.1.1516.peg
411479.10.peg.2963
411479.1.1321.peg
411479.1.peg.4050
411479.1.peg.2123
411479.1.peg.957
411479.1.2466_12.peg
411479.1.248_12.peg
411479.1.peg.1029
411479.1.peg.1435
411479.1.peg.2385
411479.1.peg.242
411479.1.1814.peg
411479.1.peg.3485
411479.1.peg.1610
411479.1.74.peg
411479.10.peg.957
411479.1.957_16.peg
411479.1.1938_11.peg
411479.1.2016_11.peg
411479.1.3734_11.peg
411479.1.3735_11.peg
411479.1.1848.peg
411479.1.1617.peg
411479.1.2704_5.peg
411479.1.2440.peg
411479.1.3812.peg
411479.1.526.peg
411479.1.3155.peg
411479.1.505.peg
411479.1.506.

In [41]:
ec_sim = add_enzyme_constraints(sim, ec_data)

Read LP format model from file /tmp/tmpaybrzej0.lp
Reading time = 0.01 seconds
: 2204 rows, 5372 columns, 18058 nonzeros


Converting to irreversible: 100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 2686/2686 [00:25<00:00, 106.57it/s]


Read LP format model from file /tmp/tmpd5r8fo89.lp
Reading time = 0.02 seconds
: 2204 rows, 7978 columns, 23936 nonzeros


Splitting isozymes:   0%|                                                                                                                                                                                                                                                                                                                          | 0/3989 [00:00<?, ?it/s]/home/mmartins/anaconda3/envs/tmcom/lib/python3.9/site-packages/cobra/core/group.py:147: UserWarning: need to pass in a list
  warn("need to pass in a list")
Adding proteins usage to reactions: 100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 5523/5523 [06:20<00:00, 14.51it/s]


### Protein pool usage

In [42]:
res = ec_sim.simulate()
print(res)

objective: 71.40464067328277
Status: OPTIMAL
Method:SimulationMethod.FBA


In [43]:
res.find('pool')

,Flux rate
Reaction ID,
prot_pool_exchange,412917.503501


### Save the model

In [45]:
write_sbml_model(ec_sim.model,f"../models/ec/ec_{model_id}.xml")

In [49]:
env = Environment.complete(ec_sim, max_uptake=1000.0, inplace=False)

env['R_prot_pool_exchange']=(0,0.8)

ec_sim.simulate(constraints=env)

ContainerAlreadyContains: Container '<optlang.container.Container object at 0x7f26f46811c0>' already contains an object with name 'fixed_objective_c5a5644a-8fbb-11ee-bda4-ac1f6bf8d98c'.